In [ ]:
modelNew = 'WF TDM v9.0 - 2023-06-30'
modelOld = 'WF TDM v8.3.2 - 2022-02-04a'

This section compares the model results between v9 and v8.3.2 for roadway volumes and transit.

## Road Volume Comparisons

::: {.content-visible when-format="html"}
The comparison between daily volumes at the segment level can be found in @fig-volume-comparison for 2019 and 2050. Decreases in volume in v9 compared to v8.3.2 are shown in blue, while increases are shown in red.
:::

::: {.content-hidden when-format="html"}
The comparison between daily volumes at the segment level can be found in @fig-pdf-volume-comparison for 2019 and 2050. Decreases in volume in v9 compared to v8.3.2 are shown in blue, while increases are shown in red.
:::

For 2019, Salt Lake and northern Davis counties display a drop in roadway volumes, most apparent on I-15. Weber, southern Davis, and Utah Counties show increases. Most of the changes are relatively minor, with the largest decreases occurring on the freeways in Salt Lake County. However, given the large daily volume for these roadways, the percent change is relatively low.

For 2050, there are decreases in volumes on I-15 in Salt Lake and northern Davis counties. Weber and northern Davis counties show overall increase in roadway volumes. Utah County shows the most change with the two Utah Lake crossings not part of the 2050 fiscally constrained scenario. The resulting drop in volumes is evident with increases on I-15.

::: {.content-visible when-format="html"}
The comparison of daily medium and heavy truck volumes is found in @fig-volume-truck-comparison for 2019 and 2050. Truck volumes decreased in the northwest portion of Salt Lake County.

::: {.panel-tabset}

### Volume Change - All Vehicles

::: {#fig-volume-comparison layout-ncol=3}

![2019](data/map_pngs/vol19-cropped.png){#fig-vol19}

![2050 Fiscally Constrained](data/map_pngs/vol50-cropped.png){#fig-vol50}

![](data/map_pngs/vol-legend-cropped.png)

Model Daily Volumes Comparison - All Vehicles (v9 vs v8.3.2)
:::

### Volume Change - Truck

::: {#fig-volume-truck-comparison layout-ncol=3}

![2019](data/map_pngs/vol19-truck-cropped.png){#fig-vol19}

![2050 Fiscally Constrained](data/map_pngs/vol50-truck-cropped.png){#fig-vol50}

![](data/map_pngs/vol-truck-legend-cropped.png)

Model Daily Volumes Comparison - Trucks (v9 vs v8.3.2)
:::

<!--

### Percent Volume Change

::: {#fig-percent-volume-comparison layout-ncol=3}

![2019](data/map_pngs/vol19_percentchange-cropped.png){#fig-vol19}

![2050](data/map_pngs/vol50_percentchange-cropped.png){#fig-vol50}

![](data/map_pngs/vol_percentchange-legend.png)

Model Daily Volumes Comparison - Percent Change (v9 vs v8.3.2)
:::
-->
:::

:::

::: {.content-hidden when-format="html"}
The comparison of daily medium and heavy truck volumes is found in @fig-pdf-volume-truck-comparison for 2019 and 2050. Truck volumes decreased in the northwest portion of Salt Lake County.

::: {#fig-pdf-volume-comparison layout-ncol=3}

![2019](data/map_pngs/vol19-cropped.png){#fig-vol19}

![2050 Fiscally Constrained](data/map_pngs/vol50-cropped.png){#fig-vol50}

![](data/map_pngs/vol-legend-cropped.png)

Model Daily Volumes Comparison - All Vehicles (v9 vs v8.3.2)
:::

::: {#fig-pdf-volume-truck-comparison layout-ncol=3}

![2019](data/map_pngs/vol19-truck-cropped.png){#fig-vol19}

![2050 Fiscally Constrained](data/map_pngs/vol50-truck-cropped.png){#fig-vol50}

![](data/map_pngs/vol-truck-legend-cropped.png)

Model Daily Volumes Comparison - Trucks (v9 vs v8.3.2)
:::

:::

## Transit Comparisons

::: {.content-visible when-format="html"}

In [ ]:
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import plotly.express as px 
import ipywidgets as widgets
import numpy as np
from scipy import special
import time
import math as msi
import sys

sys.path.insert(0, '..\..\..\..\Resources\Python-Functions')
import BigQuery
 
client = BigQuery.getBigQueryClient_TDMScenarios()

In [ ]:
# include in all scenario groups
lstIncludeInAll = ['BY','TIP']

# dataframe to create subcategories
dfModeGroups = pd.DataFrame([
    ['Walk'     ,'1: Non-Motorized'         ],
    ['Bike'     ,'1: Non-Motorized'         ],
    ['Walk'     ,'1a: Walk'                 ],
    ['Bike'     ,'1b: Bike'                 ],
    ['Auto'     ,'2: Auto'                  ],
    ['SchoolBus','3: SchoolBus'             ],
    ['LCL'      ,'4: Transit'               ],
    ['COR'      ,'4: Transit'               ],
    ['EXP'      ,'4: Transit'               ],
    ['BRT'      ,'4: Transit'               ],
    ['LRT'      ,'4: Transit'               ],
    ['CRT'      ,'4: Transit'               ],
    ['LCL'      ,'4a: Local Bus'            ],
    ['COR'      ,'4b: Core Bus'             ],
    ['EXP'      ,'4c: Express Bus'          ],
    ['BRT'      ,'4d: Bus-Rapid Transit'    ],
    ['LRT'      ,'4e: Light-Rail Transit'   ],
    ['CRT'      ,'4f: Commuter-Rail Transit'],
    ['Walk'     ,'0: Total'                 ],
    ['Bike'     ,'0: Total'                 ],
    ['Auto'     ,'0: Total'                 ],
    ['SchoolBus','0: Total'                 ],
    ['LCL'      ,'0: Total'                 ],
    ['COR'      ,'0: Total'                 ],
    ['EXP'      ,'0: Total'                 ],
    ['BRT'      ,'0: Total'                 ],
    ['LRT'      ,'0: Total'                 ],
    ['CRT'      ,'0: Total'                 ]]
,columns=('MODE','modeGroup'))
#dfModeGroups

In [ ]:
# FILTER
#strSQLWhere = ' WHERE NOT (t.scenarioID = 33)'
strSQLWhere = ''

# merge scenario data onto transit share data

# read transit summary from biq query
dfTransitSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.transit_share AS t" + strSQLWhere).to_dataframe()
#display(dfTransitSummary)

dfRouteSummary = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_summary AS t" + strSQLWhere).to_dataframe()
#display(dfRouteSummary)

# read scenarios data biq query
dfScenarios = client.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios AS t" + strSQLWhere).to_dataframe()
#display(dfScenarios)

# merge two dataframes
dfTransitSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfTransitSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge two dataframes
dfRouteSummaryWithScenarioData = pd.DataFrame.merge(dfScenarios,dfRouteSummary,on='scenarioID')
#display(dfTransitSummaryWithScenarioData)

# merge to mode groupings dataframe
dfTransitSummaryPlotData = pd.DataFrame.merge(dfTransitSummaryWithScenarioData,dfModeGroups,on='MODE')
#dfTransitSummaryPlotData

In [ ]:
#Transit Ridership Plotting Function

import math

def update_plot(tdmVersionsWithDate, scenarioGroups, modeGroups, trippurps, periods):

    data = []

    for v in tdmVersionsWithDate:
        for g in scenarioGroups:
            for m in modeGroups:

                # only do if data in dataframe since BY data is concatonated later
                if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin([g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))].shape[0]>1:

                    # data for plotting from filtered dataframe}
                    plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==v) & (dfTransitSummaryPlotData['scenarioGroup'].isin(lstIncludeInAll + [g])) & (dfTransitSummaryPlotData['modeGroup']==m) & (dfTransitSummaryPlotData['TRIPPURP'].isin(trippurps)) & (dfTransitSummaryPlotData['PERIOD'].isin(periods))]

                    #display(plotdata)

                    plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

                    # fill any NaN values with zeros
                    plotdata = plotdata.fillna(0)

                    #display(plotdata)

                    xplot = plotdata['scenarioYear']
                    yplot = plotdata['TRIPS'       ]

                    trace1 = go.Scatter(
                        x=xplot,
                        y=yplot,
                        mode='markers+lines',
                        name= v.split(' ')[2], # get version number by getting all characters before first space
                        marker=dict(size=12,
                                line=dict(width=2,
                                            color='DarkSlateGrey'))#,
                        #stackgroup='one',
                        #groupnorm='percent' # sets the normalization for the sum of the stackgroup
        #                line=dict(
        #                    shape='spline'
        #                )
                    )
                    data.append(trace1)


    layout = go.Layout(
        #title='Trips by Mode (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Trips',
            rangemode = 'tozero'#,
            #range=(0,np.null)
        ),
        xaxis=dict(
            title='Year',
            range=(2018,2051)
        ),
        width=800,
        height=550
    )
    
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))
    py.offline.iplot(fig)
py.offline.init_notebook_mode(connected=True)

In [ ]:
#Transit Share Plotting Function

import math

def update_plot_stackedarea(tdmVersionWithDate, scenarioGroup, modeGroups, trippurps, periods):

    data = []

    modeGroups = sorted(modeGroups)
    modeNames = ['Local Bus','Core Bus','Express Bus','Bus Rapid Transit','Light-Rail Transit','Commuter-Rail Transit']

    for m in modeGroups: 
        # only do if data in dataframe since BY data is concatonated later
        if dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate) &
                                    (dfTransitSummaryPlotData['scenarioGroup'     ]==scenarioGroup     ) &
                                    (dfTransitSummaryPlotData['modeGroup'         ]==m                 ) &
                                    (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)    ) &
                                    (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)      )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfTransitSummaryPlotData[(dfTransitSummaryPlotData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                (dfTransitSummaryPlotData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                (dfTransitSummaryPlotData['modeGroup'         ]==m                                     ) &
                                                (dfTransitSummaryPlotData['TRIPPURP'          ].isin(trippurps)                        ) &
                                                (dfTransitSummaryPlotData['PERIOD'            ].isin(periods)                          )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(TRIPS=('TRIPS','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['TRIPS'       ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= modeNames[modeGroups.index(m)],
                stackgroup='one',
                groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        #title=tdmVersionWithDate + ' Trips Mode Split (' + '/'.join(trippurps) + ' ' + '/'.join(periods) + ')',
        yaxis=dict(
            title='Percent of Total Trips'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=840,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)
py.offline.init_notebook_mode(connected=True)

In [ ]:
#PLOTTING FUNCTION

import math

def update_plot_stackedarea_boardings(tdmVersionWithDate, scenarioGroup):

    data = []

    modes = [4,5,6,9,7,8] #put modes in sorted order
    #modes = sorted(dfRouteSummaryWithScenarioData['MODE'].unique())
    modeNames = ['Local Bus','Core Bus','Express Bus','Bus Rapid Transit','Light-Rail Transit','Commuter-Rail Transit']

    for m in modes: 
        # only do if data in dataframe since BY data is concatonated later
        if dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate) &
                                          (dfRouteSummaryWithScenarioData['scenarioGroup'     ]==scenarioGroup     ) &
                                          (dfRouteSummaryWithScenarioData['MODE'              ]==m                 )].shape[0]>1:

            # data for plotting from filtered dataframe}
            plotdata = dfRouteSummaryWithScenarioData[(dfRouteSummaryWithScenarioData['tdmVersionWithDate']==tdmVersionWithDate                    ) &
                                                      (dfRouteSummaryWithScenarioData['scenarioGroup'     ].isin(lstIncludeInAll + [scenarioGroup])) &
                                                      (dfRouteSummaryWithScenarioData['MODE'              ]==m                                     )]

            #display(plotdata)

            plotdata = plotdata.groupby(['scenarioYear'], as_index=False).agg(BOARDINGS=('DY_BRDA','sum'))

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)

            #display(plotdata)

            xplot = plotdata['scenarioYear']
            yplot = plotdata['BOARDINGS'   ]

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name= modeNames[modes.index(m)],
                stackgroup='one',
                groupnorm='percent' # sets the normalization for the sum of the stackgroup
            )
            data.append(trace1)


    layout = go.Layout(
        #title=tdmVersionWithDate + ' Boardings by Mode',
        yaxis=dict(
            title='Percent of Total Boardings'#,
            #rangemode = 'tozero',
            #ticksuffix='%',
            #range=(0,100)
        ),
        xaxis=dict(
            title='Year'#,
            #range=(2018,2051)
        ),
        width=840,
        height=400
    )
    
    fig2 = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig2)
py.offline.init_notebook_mode(connected=True)

:::


Transit comparisons were done with ridership, trips mode share, and boardings mode share. Overall ridership increases significantly in v9, and Core Bus ridership takes a larger share of trips and boardings than in v8.3.2.

### Transit Ridership

::: {.content-visible when-format="html"}
Transit ridership in v9 compared to v8.3.2 shows significant increase in 2032, 2042, and 2050. See @fig-hy-tr-all. The total ridership in 2050 for v9 is 327,000 daily trips compared to the v8.3.2 model that showed 258,000 daily trips, which equates to 26% more trips. The additional trips is largely due to the improvements in commuter rail with increased frequency and speed together with the change in the model sensitivity to changes in headway.
:::

::: {.content-hidden when-format="html"}
Transit ridership in v9 compared to v8.3.2 shows significant increase in 2032, 2042, and 2050. See @fig-pdf-hy-tr-all. The total ridership in 2050 for v9 is 327,000 daily trips compared to the v8.3.2 model that showed 258,000 daily trips, which equates to 26% more trips. The additional trips is largely due to the improvements in commuter rail with increased frequency and speed together with the change in the model sensitivity to changes in headway.
:::

Transit ridership by modes are shown in the following set of figures. Light-Rail Transit sees an increase through 2028 and then a large decrease in 2032. This large decrease can be explained by the shift of riders from Light Rail to Core Bus routes, with a large number of core routes coming online in 2032.

::: {.content-visible when-format="html"}

::: {.panel-tabset}

<!--'4a: Local Bus','4b: Core Bus','4c: Express Bus','4d: Bus-Rapid Transit','4e: Light-Rail Transit','4f: Commuter-Rail Transit'-->

#### All Modes

In [ ]:
#| label: fig-hy-tr-all
#| fig-cap: Daily Transit Ridership - All Modes
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4: Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Commuter Rail

In [ ]:
#| label: fig-hy-tr-crt
#| fig-cap: Daily Transit Ridership - Commuter-Rail Transit
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4f: Commuter-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Light Rail

In [ ]:
#| label: fig-hy-tr-lrt
#| fig-cap: Daily Transit Ridership - Light-Rail Transit
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4e: Light-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Bus Rapid Transit

In [ ]:
#| label: fig-hy-tr-brt
#| fig-cap: Daily Transit Ridership - Bus Rapid Transit
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4d: Bus-Rapid Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Express Bus

In [ ]:
#| label: fig-hy-tr-exp
#| fig-cap: Daily Transit Ridership - Express Bus
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4c: Express Bus'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Core Bus

In [ ]:
#| label: fig-hy-tr-cor
#| fig-cap: Daily Transit Ridership - Core Bus
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4b: Core Bus'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Local Bus

In [ ]:
#| label: fig-hy-tr-lcl
#| fig-cap: Daily Transit Ridership - Local Bus
#| cap-location: margin

update_plot(
    [modelOld, modelNew], 
    ['RTP'], 
    ['4a: Local Bus'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

:::

:::


::: {.content-hidden when-format="html"}

![Daily Transit Ridership - All Modes](_pictures/pdf-hy-tr-all.png){#fig-pdf-hy-tr-all width=76%}

![Daily Transit Ridership - Commuter-Rail Transit](_pictures/pdf-hy-tr-crt.png){#fig-pdf-hy-tr-crt width=76%}

![Daily Transit Ridership - Light-Rail Transit](_pictures/pdf-hy-tr-lrt.png){#fig-pdf-hy-tr-lrt width=76%}

![Daily Transit Ridership - Bus Rapid Transit](_pictures/pdf-hy-tr-brt.png){#fig-pdf-hy-tr-brt width=76%}

![Daily Transit Ridership - Express Bus](_pictures/pdf-hy-tr-exp.png){#fig-pdf-hy-tr-exp width=76%}

![Daily Transit Ridership - Core Bus](_pictures/pdf-hy-tr-cor.png){#fig-hy-tr-cor width=76%}

![Daily Transit Ridership - Local Bus](_pictures/pdf-hy-tr-lcl.png){#fig-pdf-hy-tr-lcl width=76%}

:::

### Transit Share

::: {.content-visible when-format="html"}

A comparison of the share of trips amongst the various modes of transit was done for both Trips and Boardings.

The transit ridership trip shares by mode can be found in @fig-shr-tr-all-9 for v9 and @fig-shr-tr-all-832 for v8.3.2. The main difference in v9 trip share by mode is the large increase in Core Bus trips in 2032 from almost nothing in 2028, while in v8.3.2 the increase in Core Bus trips is spread out between 2024 and 2030. This large increase is consistent with the transit inputs into the model with a large number of Core Bus routes coming into production in 2032, replacing mostly local bus service. The new Core Buy takes most of the local bus ridership it is replacing, but also quite a lot of ridership from Light Rail Transit (Mode 7).

Transit boardings for v9 are found in @fig-brd-9 and for v8.3.2 are found in @fig-brd-832. Boardings follow the same pattern as trips, but boardings are able to differentiate between modes better than trips that are categorized hierarchically.

::: {.panel-tabset}

#### Trips - v9


In [ ]:
#| label: fig-shr-tr-all-9
#| fig-cap: Transit Trips Share by Mode - v9
#| cap-location: margin

update_plot_stackedarea(
    modelNew, 
    'RTP', 
    ['4a: Local Bus','4b: Core Bus','4c: Express Bus','4d: Bus-Rapid Transit','4e: Light-Rail Transit','4f: Commuter-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Trips - v8.3.2


In [ ]:
#| label: fig-shr-tr-all-832
#| fig-cap: Transit Trips Share by Mode - v8.3.2
#| cap-location: margin

update_plot_stackedarea(
    modelOld, 
    'RTP', 
    ['4a: Local Bus','4b: Core Bus','4c: Express Bus','4d: Bus-Rapid Transit','4e: Light-Rail Transit','4f: Commuter-Rail Transit'], 
    ['HBC', 'HBO', 'HBS', 'HBW', 'NHB'], 
    ['Pk','Ok']
)

#### Boardings - v9


In [ ]:
#| label: fig-brd-9
#| fig-cap: Transit Boardings Share by Mode - v9
#| cap-location: margin

update_plot_stackedarea_boardings(modelNew, 'RTP')

#### Boardings - v8.3.2


In [ ]:
#| label: fig-brd-832
#| fig-cap: Transit Boardings Share by Mode - v8.3.2
#| cap-location: margin

update_plot_stackedarea_boardings(modelOld, 'RTP')

:::

:::

::: {.content-hidden when-format="html"}

A comparison of the share of trips amongst the various modes of transit was done for both Trips and Boardings.

The transit ridership trip shares by mode can be found in @fig-pdf-shr-tr-all-9 for v9 and @fig-pdf-shr-tr-all-832 for v8.3.2. The main difference in v9 trip share by mode is the large increase in Core Bus trips in 2032 from almost nothing in 2028, while in v8.3.2 the increase in Core Bus trips is spread out between 2024 and 2030. This large increase is consistent with the transit inputs into the model with a large number of Core Bus routes coming into production in 2032, replacing mostly local bus service. The new Core Buy takes most of the local bus ridership it is replacing, but also quite a lot of ridership from Light Rail Transit (Mode 7).

Transit boardings for v9 are found in @fig-pdf-brd-9 and for v8.3.2 are found in @fig-pdf-brd-832. Boardings follow the same pattern as trips, but boardings are able to differentiate between modes better than trips that are categorized hierarchically.

![Transit Trips Share by Mode - v9](_pictures/pdf-shr-tr-all-9.png){#fig-pdf-shr-tr-all-9 width=76%}

![Transit Trips Share by Mode - v8.3.2](_pictures/pdf-shr-tr-all-832.png){#fig-pdf-shr-tr-all-832 width=76%}

![Transit Boardings Share by Mode - v9](_pictures/pdf-brd-9.png){#fig-pdf-brd-9 width=76%}

![Transit Boardings Share by Mode - v8.3.2](_pictures/pdf-brd-832.png){#fig-pdf-brd-832 width=76%}

:::


#### Commuter Rail Station Boardings

::: {.content-visible when-format="html"}

The comparison of base year (2019) station-level boardings for commuter-rail transit (CRT) is found in @fig-fr-brd. CRT boardings were found to be higher than observed for Davis County and lower than observed for Utah County. An adjustment of 5 additional minutes to in-vehicle-time for trips to/from Davis County and 5 fewer minute to in-vehicle-time for Utah County was made to attempt to bring the model more in-line with observations.

Additional investigation was conducted into why Provo and Lehi were particularly low in the model. The findings did not turn up any obvious errors in the transit or model network. So, the conclusion is that further adjustments to CRT will be possible in the Mode Choice Update project that is currently being undertaken for the next release of the model.


In [ ]:
#| label: fig-fr-brd
#| fig-cap: 2019 Daily CRT Boardings by Station - Model vs Observed
#| cap-location: margin

boardings_barchart = pd.read_csv(r'data\frontrunner-summary-comparison-v832-v9.csv')
import plotly.express as px
df = px.data.tips()
fig = px.histogram(boardings_barchart, x="Stop_Name", y="Boardings", text_auto='.2s',
             color='Model_Name', barmode='group',
             height=400)
fig.update_layout(
    xaxis_title="Station Name",
    yaxis_title="Average Boardings",
    legend_title="Model Version"
)
#fig.update_xaxes(tickangle=90)
fig.show()

:::

::: {.content-hidden when-format="html"}

The comparison of base year (2019) station-level boardings for commuter-rail transit (CRT) is found in @fig-pdf-fr-brd. CRT boardings were found to be higher than observed for Davis County and lower than observed for Utah County. An adjustment of 5 additional minutes to in-vehicle-time for trips to/from Davis County and 5 fewer minute to in-vehicle-time for Utah County was made to attempt to bring the model more in-line with observations.

Additional investigation was conducted into why Provo and Lehi were particularly low in the model. The findings did not turn up any obvious errors in the transit or model network. So, the conclusion is that further adjustments to CRT will be possible in the Mode Choice Update project that is currently being undertaken for the next release of the model.

![2019 Daily CRT Boardings by Station - Model vs Observed](_pictures/pdf-fr-brd.png){#fig-pdf-fr-brd width=76%}

:::